In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

# New Section

In [ ]:
!ls spark-3.0.0-bin-hadoop3.2

ls: cannot access 'spark-3.0.0-bin-hadoop3.2': No such file or directory


In [ ]:
!java -version

openjdk version "11.0.16" 2022-07-19
OpenJDK Runtime Environment (build 11.0.16+8-post-Ubuntu-0ubuntu118.04)
OpenJDK 64-Bit Server VM (build 11.0.16+8-post-Ubuntu-0ubuntu118.04, mixed mode, sharing)


In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 50 kB/s 
     |████████████████████████████████| 199 kB 57.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=cf871145df71f15d8d28406620446c439e20ae9d8d9320acdf7eed7f66de9b85
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
#from pyspark.sql import SparkSession

#spark = SparkSession.Builder.getOrCreate(self)
#print(spark)

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
print(spark)
print(spark.version)

3.3.0


In [ ]:
#import pandas as pd

#url = 'https://drive.google.com/file/d/170fKKzEg__asJ-QX3ufyswVvHJNY3hdJ/view?usp=sharing'
#id_file = url.split('/')[-2]
#baseurl = 'https://drive.google.com/uc?id=' + id_file
df = spark.read.csv('/content/drive/MyDrive/kuliah/Smester 5/big data lanjut/p1/flights_small.csv', header=True, inferSchema=True)
#dataset = pd.read_csv(baseurl)
#dataset.head()
df.write.saveAsTable("flights")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
print(spark.catalog.listTables())


[Table(name='flights', database='default', description=None, tableType='MANAGED', isTemporary=False)]


In [ ]:
query = "FROM flights SELECT * LIMIT 10"
flights10 = spark.sql(query)
flights10.show()

#test = "FROM flights SELECT COUNT(FROM flights SELECT dep_delay)"

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [ ]:
flights_count = spark.sql("FROM flights SELECT COUNT(*) as Total_rows")
flights_count.show()

+----------+
|Total_rows|
+----------+
|     10000|
+----------+



In [ ]:

query = "SELECT origin, dest, COUNT(*) as N FROM flights GROUP BY origin, dest"


flight_counts = spark.sql(query)
pd_counts = flight_counts.toPandas()

print(pd_counts.head())
pd_counts.head()

  origin dest    N
0    SEA  RNO    8
1    SEA  DTW   98
2    SEA  CLE    2
3    SEA  LAX  450
4    PDX  SEA  144


,origin,dest,N
0,SEA,RNO,8
1,SEA,DTW,98
2,SEA,CLE,2
3,SEA,LAX,450
4,PDX,SEA,144


In [ ]:
pd_counts.to_csv('flight_counts.csv')
spark.catalog.listTables()

[Table(name='flights', database='default', description=None, tableType='MANAGED', isTemporary=False)]

In [ ]:
flights = spark.table('flights')
flights = flights.withColumn('duration_hrs', flights.air_time / 60)
flights.show()
query = "SELECT duration_hrs FROM flights"
#durationHours = spark.sql(query)


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|              1.85|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|1.3833333333333333|
|2014|    3|  9|     754|  

**P2**
query inside spark database

In [ ]:
df.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [ ]:
long_flights1 = df.filter("distance > 1000")
long_flights2 = df.filter(df.distance > 1000)

In [ ]:
long_flights1.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|     135|    1050|  12|    36|
|2014|   11| 19|    1812|       -3|    2352|       -4|     AS| N564AS|    26|   SEA| ORD|     198|    1721|  18|    12|
|2014|    8|  3|    1120|        0|    1415|        2|     AS| N305AS|   656|   SEA| PHX|     154|    1107|  11|    20|
|2014|   11| 12|    2346|       -4|     217|      -28|     AS| N765AS|   121|   SEA| ANC|     183|    1448|  23|    46|
|2014|    8| 11|    1017|       -3|    1

In [ ]:
selected1 = df.select('tailnum','origin','dest')
selected1.show()

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N846VA|   SEA| LAX|
| N559AS|   SEA| HNL|
| N847VA|   SEA| SFO|
| N360SW|   PDX| SJC|
| N612AS|   SEA| BUR|
| N646SW|   PDX| DEN|
| N422WN|   PDX| OAK|
| N361VA|   SEA| SFO|
| N309AS|   SEA| SAN|
| N564AS|   SEA| ORD|
| N323AS|   SEA| LAX|
| N305AS|   SEA| PHX|
| N433AS|   SEA| LAS|
| N765AS|   SEA| ANC|
| N713AS|   SEA| SFO|
| N27205|   PDX| SFO|
| N626AS|   SEA| SMF|
| N8634A|   SEA| MDW|
| N597AS|   SEA| BOS|
| N215AG|   PDX| BUR|
+-------+------+----+
only showing top 20 rows



In [ ]:
select2 = selected1.filter(df.distance > 1000)
select2.show()

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N559AS|   SEA| HNL|
| N309AS|   SEA| SAN|
| N564AS|   SEA| ORD|
| N305AS|   SEA| PHX|
| N765AS|   SEA| ANC|
| N8634A|   SEA| MDW|
| N597AS|   SEA| BOS|
| N127UW|   SEA| PHL|
| N395HA|   SEA| OGG|
| N461AS|   SEA| SAN|
| N590JB|   SEA| ANC|
| N512AS|   SEA| LIH|
| N407AS|   SEA| MCO|
| N824DN|   SEA| ATL|
| N402AS|   SEA| ANC|
| N589AS|   SEA| ABQ|
| N68805|   SEA| IAH|
| N662AW|   SEA| PHX|
| N309US|   PDX| MSP|
| N172US|   SEA| PHX|
+-------+------+----+
only showing top 20 rows



nested filter 

In [ ]:
selected3 = selected1.filter(df.distance > 1000).filter(df.dest == "OGG").filter(df.origin =="SEA")
selected3.show()

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N395HA|   SEA| OGG|
| N589AS|   SEA| OGG|
| N537AS|   SEA| OGG|
| N577AS|   SEA| OGG|
| N562AS|   SEA| OGG|
| N562AS|   SEA| OGG|
| N579AS|   SEA| OGG|
| N585AS|   SEA| OGG|
| N711ZX|   SEA| OGG|
| N393HA|   SEA| OGG|
| N373HA|   SEA| OGG|
| N590AS|   SEA| OGG|
| N535AS|   SEA| OGG|
| N558AS|   SEA| OGG|
| N711ZX|   SEA| OGG|
| N566AS|   SEA| OGG|
| N386HA|   SEA| OGG|
| N384HA|   SEA| OGG|
| N597AS|   SEA| OGG|
| N385HA|   SEA| OGG|
+-------+------+----+
only showing top 20 rows



In [ ]:
avg_speed = (df.distance/(df.air_time/60)).alias("avg_speed")
speed1 = df.select("origin", "dest", "tailnum",avg_speed)
speed1.show()
#or you can use

speed2 = df.selectExpr("origin", "dest", "tailnum", "distance/(air_time/60) as avg_speed")
speed2.show()

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
|   PDX| DEN| N646SW|491.40495867768595|
|   PDX| OAK| N422WN|             362.0|
|   SEA| SFO| N361VA| 415.7142857142857|
|   SEA| SAN| N309AS| 466.6666666666667|
|   SEA| ORD| N564AS| 521.5151515151515|
|   SEA| LAX| N323AS| 440.3076923076923|
|   SEA| PHX| N305AS|431.29870129870125|
|   SEA| LAS| N433AS| 409.6062992125984|
|   SEA| ANC| N765AS|474.75409836065575|
|   SEA| SFO| N713AS| 315.8139534883721|
|   PDX| SFO| N27205| 366.6666666666667|
|   SEA| SMF| N626AS|477.63157894736844|
|   SEA| MDW| N8634A|481.38888888888886|
|   SEA| BOS| N597AS| 516.4137931034483|
|   PDX| BUR| N215AG| 441.6216216216216|
+------+----+-------+------------------+
only showing top

Agregate 

In [ ]:
#convert to float
flights  = df.withColumn('distance', df.distance.cast('float'))
flights  = flights.withColumn('air_time', flights.air_time.cast('float'))

flights.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|   132.0|   954.0|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|   360.0|  2677.0|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|   111.0|   679.0|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|    83.0|   569.0|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|   127.0|   937.0|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [ ]:
df.groupBy().min("distance").show()
#combine filter and groupby
df.filter(df.dest =="LAX").groupBy().min("distance").show()

+-------------+
|min(distance)|
+-------------+
|         93.0|
+-------------+

+-------------+
|min(distance)|
+-------------+
|        834.0|
+-------------+



In [ ]:
# get max airfly time
df.groupby().max("distance").show()
df.groupby().min("distance").show()

+-------------+
|max(distance)|
+-------------+
|       2724.0|
+-------------+

+-------------+
|min(distance)|
+-------------+
|         93.0|
+-------------+



In [ ]:
# average with origin = PDX

flights.filter(flights.origin == "PDX").groupBy().avg('air_time').show()

+------------------+
|     avg(air_time)|
+------------------+
|137.11543248288737|
+------------------+



In [ ]:
#hitung rata rata waktu penerbangan di mana jaraknya > 1000
flights.filter(flights.distance > 1000).groupBy().avg('air_time').show()

+------------------+
|     avg(air_time)|
+------------------+
|209.34971098265896|
+------------------+



In [ ]:
#hitung waktu tercepat penerbangan dimana jaraknya > 1000
flights.filter(flights.distance > 1000).groupBy().min('air_time').show()

+-------------+
|min(air_time)|
+-------------+
|        101.0|
+-------------+

